In [3]:
import sys
sys.path.append('/Users/jinchen/Desktop/DSE/IntroML/Project/project-product-price-prediction')

In [4]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
matplotlib.rcParams['figure.figsize'] = (15, 15)
plt.rcParams['figure.constrained_layout.use'] = True


# feature selection
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_regression # F-value between label/feature for regression tasks.

from sklearn.metrics import mean_squared_log_error as msle


from final.random_sampling.even_sample_brand import stratified_sampling_by_brand
from final.random_sampling.even_sample_category import stratified_sampling_by_category
from final.random_sampling.even_sample_by_price_range import stratified_sampling_by_price
from final.feature_extraction.text_vectorization import encode_categories,encode_string_column

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Get sample df

In [3]:
df = stratified_sampling_by_brand(df, file_dir="data",number_samples = 10000,
                                               replace = False, save_sample_df = Fasle)


(93550, 59)

In [4]:
df.head(5)

,train_id,item_condition_id,brand_name,price,shipping,c1,c2,c3,clean_name,clean_description,...,brand_nameLowerRatio,brand_nameUpperRatio,brand_nameAvgWordLen,nameStopWordCount,namePunctuationCount,nameSpecialCount,item_descriptionStopWordCount,item_descriptionPunctuationCount,item_descriptionSpecialCount,price_bin
0,804721,3,brighton,22.0,0,woman,woman,woman,brighton galadriel necklace bracelet,brighton galadriel pendant necklace and matchi...,...,0.000845,0.0,0.001017,0.0,0.000919,0.002179,0.001533,0.000511,0.000397,"(20, 25]"
1,1190931,3,express,8.0,0,woman,woman,woman,express dress pants,size 3 4 womens great condition,...,0.000845,0.0,0.000890,0.0,0.000000,0.000000,0.000000,0.000073,0.000000,"(5, 10]"
2,114352,1,hanes,11.0,1,men,men,men,men s black hoodie,hanes men s comfortblend pullover hoodie black...,...,0.000845,0.0,0.000636,0.0,0.000919,0.000000,0.000064,0.000292,0.000198,"(10, 15]"
3,376371,1,philip,41.0,0,kid,kid,kid,philips sonicare toothbrush,electric toothbrush two shades whiter in two w...,...,0.000845,0.0,0.000763,0.0,0.000000,0.000000,0.000064,0.000146,0.000066,"(40, 50]"
4,873087,3,vtech,20.0,0,kid,kid,kid,vtech smart shots sports center toy,good used condition all sounds and lights work...,...,0.000845,0.0,0.000636,0.0,0.000000,0.000000,0.001341,0.000511,0.000860,"(15, 20]"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93550 entries, 0 to 93549
Data columns (total 59 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   train_id                          93550 non-null  int64  
 1   item_condition_id                 93550 non-null  int64  
 2   brand_name                        93550 non-null  object 
 3   price                             93550 non-null  float64
 4   shipping                          93550 non-null  int64  
 5   c1                                93550 non-null  object 
 6   c2                                93550 non-null  object 
 7   c3                                93550 non-null  object 
 8   clean_name                        93550 non-null  object 
 9   clean_description                 93550 non-null  object 
 10  clean_nameWordCount               93550 non-null  float64
 11  clean_nameLength                  93550 non-null  float64
 12  clea

# Train Test Split

In [6]:
Y = np.log1p(df['price'])
df.drop(['price','price_bin'], axis=1, inplace=True)
df.head(2)

,train_id,item_condition_id,brand_name,shipping,c1,c2,c3,clean_name,clean_description,clean_nameWordCount,...,brand_nameLowerCount,brand_nameLowerRatio,brand_nameUpperRatio,brand_nameAvgWordLen,nameStopWordCount,namePunctuationCount,nameSpecialCount,item_descriptionStopWordCount,item_descriptionPunctuationCount,item_descriptionSpecialCount
0,804721,3,brighton,0,woman,woman,woman,brighton galadriel necklace bracelet,brighton galadriel pendant necklace and matchi...,0.000682,...,0.000789,0.000845,0.0,0.001017,0.0,0.000919,0.002179,0.001533,0.000511,0.000397
1,1190931,3,express,0,woman,woman,woman,express dress pants,size 3 4 womens great condition,0.000511,...,0.000690,0.000845,0.0,0.000890,0.0,0.000000,0.000000,0.000000,0.000073,0.000000


In [7]:
train_df, test_df , y_train, y_test = train_test_split(df, Y, test_size=0.2, random_state=12342)
print('Train size: %s, Test size: %s'%(train_df.shape, test_df.shape))

Train size: (74840, 57), Test size: (18710, 57)


# Vectorization

In [19]:
train_cat_features, train_cat_features_name = encode_categories(train_df, 
                                                                columns = ['c1','c2','c3','brand_name'], 
                                                                min_df = 10, print_progress=True)


Size of vectorization features of c1 is 15
Size of vectorization features of c2 is 15
Size of vectorization features of c3 is 15
Size of vectorization features of brand_name is 1008
Shape of total vectorization features of ['c1', 'c2', 'c3', 'brand_name'] is (74840, 1053)


In [9]:
train_str_features, train_str_features_name = encode_string_column(train_df, 
                                                                   columns=['clean_name', 'clean_description'],
                                                                   min_df=10, max_features=15000, 
                                                                   print_progress=True)

Size of vectorization features of clean_name is 6150
Size of vectorization features of clean_description is 15000
Shape of vectorization features of ['clean_name', 'clean_description'] is (74840, 21150)


In [24]:
test_cat_features, test_cat_features_name = encode_categories(test_df, 
                                                                columns = ['c1','c2','c3','brand_name'], 
                                                                min_df = 10, print_progress=True)



Size of vectorization features of c1 is 14
Size of vectorization features of c2 is 14
Size of vectorization features of c3 is 14
Size of vectorization features of brand_name is 1008
Shape of total vectorization features of ['c1', 'c2', 'c3', 'brand_name'] is (18710, 1050)


In [25]:
test_str_features, test_str_features_name = encode_string_column(test_df, 
                                                                   columns=['clean_name', 'clean_description'],
                                                                   min_df=10, max_features=15000, 
                                                                   print_progress=True)

Size of vectorization features of clean_name is 2063
Size of vectorization features of clean_description is 9130
Shape of vectorization features of ['clean_name', 'clean_description'] is (18710, 11193)


# Combine all features

In [15]:
other_columns = list(train_df.select_dtypes([np.number]).columns)
other_columns.remove('train_id')
other_columns

['item_condition_id',
 'shipping',
 'clean_nameWordCount',
 'clean_nameLength',
 'clean_nameUpperCount',
 'clean_nameLowerCount',
 'clean_nameLowerRatio',
 'clean_nameUpperRatio',
 'clean_nameAvgWordLen',
 'clean_descriptionWordCount',
 'clean_descriptionLength',
 'clean_descriptionUpperCount',
 'clean_descriptionLowerCount',
 'clean_descriptionLowerRatio',
 'clean_descriptionUpperRatio',
 'clean_descriptionAvgWordLen',
 'c1WordCount',
 'c1Length',
 'c1UpperCount',
 'c1LowerCount',
 'c1LowerRatio',
 'c1UpperRatio',
 'c1AvgWordLen',
 'c2WordCount',
 'c2Length',
 'c2UpperCount',
 'c2LowerCount',
 'c2LowerRatio',
 'c2UpperRatio',
 'c2AvgWordLen',
 'c3WordCount',
 'c3Length',
 'c3UpperCount',
 'c3LowerCount',
 'c3LowerRatio',
 'c3UpperRatio',
 'c3AvgWordLen',
 'brand_nameWordCount',
 'brand_nameLength',
 'brand_nameUpperCount',
 'brand_nameLowerCount',
 'brand_nameLowerRatio',
 'brand_nameUpperRatio',
 'brand_nameAvgWordLen',
 'nameStopWordCount',
 'namePunctuationCount',
 'nameSpecialCoun

In [16]:
train_other_features = train_df[other_columns].values
test_other_features = test_df[other_columns].values
print('Train other features size: %s, Test other features size: %s'%(train_other_features.shape, test_other_features.shape))

Train other features size: (74840, 50), Test other features size: (18710, 50)


In [ ]:
all_train = np.hstack((train_cat_features, train_str_features, train_other_features))
all_test = np.hstack((test_cat_features, test_str_features, test_other_features))
print('Train features size: %s, Test features size: %s'%(all_train.shape, 
                                                         all_test.shape))

# Feature Selection

In [ ]:
estimator = Ridge(solver="sag",alpha=4.5)
selector = RFECV(estimator, step=1, cv=5, min_features_to_select = 1000)
selector = selector.fit(all_train, y_train)


In [ ]:
selector.support_

In [ ]:
selector.ranking_

In [ ]:
select_train = all_train[selector.support_]
select_train.shape

In [ ]:
select_test = all_test[selector.support_]
select_test.shape

# Hyperparameter Tuning

## Random Forest Regressor

# Train with best hyparameter